In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-signature-data/test/test/061/061_015.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_006.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_021.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_005.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_003.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_011.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_023.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_013.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_012.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_024.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_009.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_008.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_014.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_018.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_002.jpeg
/kaggle/input/arabic-signature-data/test/test/061/061_0

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

base_dir = "/kaggle/input/arabic-signature-data"  
train_dir = os.path.join(base_dir, "train", "train")
test_dir = os.path.join(base_dir, "test", "test")

img_size = (128, 128)  
batch_size = 32

def load_data(data_dir):
    images = []
    labels = []
    classes = os.listdir(data_dir)  
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        label = 0 if "_forg" in class_name else 1  
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            image = tf.io.read_file(img_path)
            image = tf.image.decode_image(image, channels=3)
            image = tf.image.resize(image, img_size) / 255.0  
            images.append(image)
            labels.append(label)
    
    return np.array(images), np.array(labels)

train_images, train_labels = load_data(train_dir)

X_train, X_val, y_train, y_val = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

test_images, test_labels = load_data(test_dir)


In [4]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # Binary classification: Genuine (1) or Forged (0)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=batch_size)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 466ms/step - accuracy: 0.5358 - loss: 0.7611 - val_accuracy: 0.6980 - val_loss: 0.5867
Epoch 2/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 34s 455ms/step - accuracy: 0.6485 - loss: 0.6190 - val_accuracy: 0.7064 - val_loss: 0.5327
Epoch 3/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 34s 453ms/step - accuracy: 0.7297 - loss: 0.5410 - val_accuracy: 0.7164 - val_loss: 0.5072
Epoch 4/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 41s 456ms/step - accuracy: 0.7348 - loss: 0.5215 - val_accuracy: 0.7903 - val_loss: 0.4345
Epoch 5/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 34s 457ms/step - accuracy: 0.7376 - loss: 0.5527 - val_accuracy: 0.7668 - val_loss: 0.4951
Epoch 6/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 34s 460ms/step - accuracy: 0.7750 - loss: 0.4731 - val_accuracy: 0.8138 - val_loss: 0.3900
Epoch 7/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 34s 454ms/step - accuracy: 0.6886 - loss: 0.5892 - val_accuracy: 0.7030 - val_loss: 0.5480
Epoch 8/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 41s 458ms/step - accuracy: 0.6618 - loss: 0.6000 - val_accu

In [5]:
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  
    layers.Dense(128, activation='relu'),  
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 230s 3s/step - accuracy: 0.4993 - loss: 0.8165 - val_accuracy: 0.6594 - val_loss: 0.6341
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 299s 4s/step - accuracy: 0.6510 - loss: 0.6292 - val_accuracy: 0.6544 - val_loss: 0.6156
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 229s 3s/step - accuracy: 0.6688 - loss: 0.6140 - val_accuracy: 0.7265 - val_loss: 0.5665
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.6957 - loss: 0.5709 - val_accuracy: 0.7534 - val_loss: 0.5371
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 265s 4s/step - accuracy: 0.7140 - loss: 0.5647 - val_accuracy: 0.7651 - val_loss: 0.5086
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 229s 3s/step - accuracy: 0.7304 - loss: 0.5385 - val_accuracy: 0.7735 - val_loss: 0.4924
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 265s 4s/step - accuracy: 0.7314 - loss: 0.5296 - val_accuracy: 0.7802 - val_loss: 0.4815
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 266s 4s/step - accurac

In [6]:
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  
    layers.Dense(128, activation='relu'),  
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax') 
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 47s 528ms/step - accuracy: 0.6305 - loss: 0.7681 - val_accuracy: 0.7466 - val_loss: 0.5163
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 40s 533ms/step - accuracy: 0.7857 - loss: 0.4809 - val_accuracy: 0.7752 - val_loss: 0.4699
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 491ms/step - accuracy: 0.8152 - loss: 0.4073 - val_accuracy: 0.8171 - val_loss: 0.4282
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 493ms/step - accuracy: 0.8156 - loss: 0.4118 - val_accuracy: 0.8339 - val_loss: 0.3959
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 491ms/step - accuracy: 0.8491 - loss: 0.3466 - val_accuracy: 0.8490 - val_loss: 0.3835
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 40s 535ms/step - accuracy: 0.8492 - loss: 0.3384 - val_accuracy: 0.8523 - val_loss: 0.3825
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 494ms/step - accuracy: 0.8727 - loss: 0.3075 - val_accuracy: 0.8507 - val_loss: 0.3678
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 41s 498m

In [7]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

# Add custom classification layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  
    layers.Dense(128, activation='relu'), 
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32)

# Evaluate on test data
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

Epoch 1/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 47s 527ms/step - accuracy: 0.5701 - loss: 1.0160 - val_accuracy: 0.7399 - val_loss: 0.5282
Epoch 2/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 493ms/step - accuracy: 0.7401 - loss: 0.5225 - val_accuracy: 0.7735 - val_loss: 0.4651
Epoch 3/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 493ms/step - accuracy: 0.7838 - loss: 0.4358 - val_accuracy: 0.8054 - val_loss: 0.4265
Epoch 4/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 40s 533ms/step - accuracy: 0.8164 - loss: 0.4118 - val_accuracy: 0.8121 - val_loss: 0.4144
Epoch 5/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 492ms/step - accuracy: 0.8297 - loss: 0.3860 - val_accuracy: 0.8255 - val_loss: 0.3991
Epoch 6/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 40s 533ms/step - accuracy: 0.8305 - loss: 0.3741 - val_accuracy: 0.8440 - val_loss: 0.3653
Epoch 7/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 489ms/step - accuracy: 0.8648 - loss: 0.3319 - val_accuracy: 0.8339 - val_loss: 0.3742
Epoch 8/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 37s 493ms/step - accuracy: 0.8808 - loss: 0.2825 - val_accu